## Introduction


In this notebook, we use the Bluemix CLI tools to create a new IBM Analytics Engine instance.

*Prerequisites:* 
- You have worked through the notebook `examples/CLI/CLI_Setup.ipynb`

*Recommended:*
- You have worked through the notebook `examples/CLI/Provision_IAE.ipynb`

---

First let's set this notebook to use the full browser width.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

We can read some variables saved when we ran the notebook `examples/CLI/CLI_Setup.ipynb` to configure our choosen api, org and space

In [ ]:
import json

with open('../../target_endpoint.json') as data_file:    
    target_endpoint = json.load(data_file)
    
api   = target_endpoint['api']
org   = target_endpoint['org']
space = target_endpoint['space']
   
print('api: '   + api)
print('org: '   + org)
print('space: ' + space)

Create a file `../../cos_s3_endpoint.json` with your COS credentials.  The file format should be:

```
{
   "S3_ACCESS_KEY": "<AccessKey-changeme>",
   "S3_ENDPOINT":   "<EndPoint-changeme>",
   "S3_SECRET_KEY": "<SecretKey-changeme>"
}
```

In [ ]:
with open('../../cos_s3_endpoint.json') as data_file:    
    cos_s3_endpoint = json.load(data_file)
    
S3_ACCESS_KEY = cos_s3_endpoint['S3_ACCESS_KEY']
S3_ENDPOINT   = cos_s3_endpoint['S3_ENDPOINT']
S3_SECRET_KEY = cos_s3_endpoint['S3_SECRET_KEY']

print('S3 Endpoint: ' + S3_ENDPOINT)

We can login to Bluemix using the Bluemix CLI

In [ ]:
! bx login --apikey @../../apiKey.json -a {api} -o {org} -s {space}

In [ ]:
custom_script = { 
    "num_compute_nodes": 1, 
    "hardware_config": "Standard", 
    "software_package": "ae-1.0-spark",
    "customization": [{
        "name": "action1",
        "type": "bootstrap",
        "script": {
            "source_type": "http",
            "script_path": "http://raw.githubusercontent.com/snowch/IBM_Analytics_Engine_Examples/master/scripts/COS_S3.sh"
            },
        "script_params": [S3_ACCESS_KEY, S3_ENDPOINT, S3_SECRET_KEY]
        }]
}

with open('custom_script.json', 'w') as f:
    f.write(json.dumps(custom_script))

We can then attempt to create an IBM Analytics Engine Instance.

In [ ]:
! bx cf create-service IBMAnalyticsEngine Standard 'myiaeinstance' -c custom_script.json

Note the output from above.

TODO - add recommendations for errors.

If all went ok, the CLI should suggest running `cf service myiaeinstance` to check the provisioning status. 

Let's do that now.

In [ ]:
! bx cf service myiaeinstance

---

## Debugging 

TODO 

In [ ]:
! bx cf services

In [ ]:
! bx cf delete-service -f 'myiaeinstance'

In [ ]:
!cf delete-service-key -f 'IBM Analytics Engine-pj' 'Credentials-1'